In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import random
from operator import add

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [7]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param W {J} >=0;   # number of worker of doing the job
param K {I} >=0;   # capacity of machine 
param R {I} >=0;   # quality factor of machine 
param S {I} >=0;   # speed of machine 

var X {I,J} >=0 ;   # Decision var 1
var Y {J}   >=0 ;   # Decision var 2

maximize Benefit: sum {i in I, j in J} R[i] * B[j] * X[i,j];


s.t. CoWork   {        j in J}: Y[j] * W[j] = sum {i in I} X[i,j];

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= S[i] * K[i];
s.t. Arrange1 {i in I, j in J}:                     X[i,j] <= 1;
s.t. Arrange2 {i in I, j in J}:                     X[i,j] >= 0;
s.t. Arrange3 {        j in J}:                     Y[  j] <= 1;
s.t. Arrange4 {        j in J}:                     Y[  j] >= 0;

''')

## Step 5: Define the initial data

In [8]:
machine_job  = [(5,50),(15,50),(15,500),(50,500)]
cowork       = ['O','F','M']     # 
capacity     = ['N','L','T']     #  
relation_bc  = ['L','X','A','R'] #

In [9]:
# batch 要做成:
# [ num_machine, num_job, cost, benefit, capacity ]

it_n = 100

def batch (num_machine, num_job, relation_bc, capacity, cowork):
    
    m = num_machine
    n = num_job
    
    list_r1 = []    # 內含 100組 m個 machine的 quality factor   組合
    list_s1 = []    # 內含 100組 m個 machine的 speed            組合
    
    list_c1 = []    # 內含 100組 n個 job的 cost                 組合
    list_w1 = []    # 內含 100組 n個 job的 worker               組合
    list_b1 = []    # 內含 100組 n個 job的 benefit              組合
    
    list_k1 = []    # 內含 100組 由 n個 job 加總算出的 k
    
    for h in range(it_n):
        
        list_r2 = []   # 內含 m個 machine的 quality factor
        list_s2 = []   # 內含 m個 machine的 speed        
    
        list_c2 = []   # 內含 n個 job的 cost
        list_w2 = []   # 內含 n個 job的 worker
        list_b2 = []   # 內含 n個 job的 benefit 
                    
        for i in range(m):
            
            r = (random.random())*9 + 1   # 1-10之間亂數  
            s = (random.random())*9 + 1   # 1-10之間亂數
            list_r2.append(r)
            list_s2.append(s)
            
        for j in range(n):
                
            c = int(random.randint(1,50))
            list_c2.append(c)

            if   relation_bc == 'L':
                b = c

            elif relation_bc == 'X':
                b = c**2

            elif relation_bc == 'A':
                b = c**(1/2)

            elif relation_bc == 'R':
                b = random.randint(0,50) 
            
            list_b2.append(b)
            
            
            if   cowork == 'M': # most
                q = m
                
            elif cowork == 'F': # some
                q = int(m*1/2)
                
            elif cowork == 'O': # few
                q = 1
                
            w = random.randint(1, q)
            list_w2.append(w)
            
        if   capacity =='N':
            k = 100000 # no capacity

        elif capacity =='L':
            k = int( sum(list_c2) / m )

        elif capacity =='T':
            k = int( sum(list_c2) * (0.75) / m )

        list_r1.append(list_r2)
        list_s1.append(list_s2)
        list_c1.append(list_c2)
        list_w1.append(list_w2)
        list_b1.append(list_b2)
        list_k1.append(k)
    
    return  m, n, list_c1, list_w1, list_b1, list_k1, list_r1, list_s1

In [26]:
# Test OK! 不要在正式 run 100組時  print，會很慢 >"<
'''
instance = batch( 5, 20, 'A', 'T', 'M' )
print(instance[2])
print(instance[3])
print(instance[4])

print(instance[6])
print(instance[7])
'''

[[19, 25, 39, 16, 26, 3, 49, 33, 48, 45, 23, 10, 28, 14, 24, 49, 18, 37, 17, 50]]
[[1, 2, 1, 5, 3, 1, 2, 5, 5, 1, 4, 2, 4, 5, 4, 2, 2, 5, 2, 1]]
[[4.358898943540674, 5.0, 6.244997998398398, 4.0, 5.0990195135927845, 1.7320508075688772, 7.0, 5.744562646538029, 6.928203230275509, 6.708203932499369, 4.795831523312719, 3.1622776601683795, 5.291502622129181, 3.7416573867739413, 4.898979485566356, 7.0, 4.242640687119285, 6.082762530298219, 4.123105625617661, 7.0710678118654755]]


## Step 6: Model Build-up & Solution

In [10]:
# 迭代 instance

TotalBenefit_ijk_sn = []
instance_list = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]

    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for l in cowork:
        for k in capacity:    
            for j in relation_bc:
                                
                instance = batch( i[0], i[1], j, k, l )
                instance_list.append(instance)

                TotalBenefit_1 = []
                TotalBenefit_2 = []

                for p in range(it_n):    

                    if k == 'N':
                        ca_adj = instance[7][p]
                    else:
                        ca_adj = [1] * num_machine                
                    
                    # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均

                    # amplpy DataFrame
                    ampl.setData(DataFrame(
                    index=[('J', job)], 
                    columns=[
                        ('B', instance[4][p]), 
                        ('W', instance[3][p]), 
                        ('C', instance[2][p])
                    ]
                    ))

                    # Pandas DataFrame
                    df = pd.DataFrame({
                        'K': [instance[5][p]]*num_machine,
                        'R':  instance[6][p],
                        'S':  list(map(lambda a, b: a / b, instance[7][p], ca_adj))
                    }, 
                        index = machine
                    )
                    ampl.setData(DataFrame.fromPandas(df))

                    
                    # Solve the problem
                    ampl.solve()

                    '''
                    # Create a DataFrame with Decision variables
                    Var = ampl.getVariable('X').getValues().toPandas()
                    print(Var)
                    '''

                    # Display the objective value
                    TotalBnft_1 = ampl.getObjective('Benefit')
                    TotalBnft_1 = TotalBnft_1.value()
                    TotalBenefit_1 = np.append(TotalBenefit_1, TotalBnft_1) # numpy.ndarray


                    # ====== Model 2 ======= : 一組一組用 CHBF_W 算，每種senario要算 100組 再平均

                    # 將 job 按照 cost (workload) 排序

                    list_c = instance[2][p]
                    list_w = instance[3][p]
                    list_b = instance[4][p]

                    list_cwb = list(zip(list_c, list_w, list_b)) # [3,5] [4,2] [8,1] -> [(3,4,8),(5,2,1)]
                    s_cwb    = sorted(list_cwb, reverse = True)  # [(3,4,8),(5,2,1)] -> [(5,2,1),(3,4,8)]
                    o_cwb    = list(zip(*s_cwb))                 # [(5,2,1),(3,4,8)] -> [(5,3),(2,4),(1,8)]
                    
                    list_c  = list(o_cwb[0])                    # [5,3]
                    list_w  = list(o_cwb[1])                    # [2,4]
                    list_b  = list(o_cwb[2])                    # [1,8]
                    
                    # 將 machine 按照 SiRi 排序
                    
                    list_r = instance[6][p]
                    list_s = list(map(lambda a, b: a / b, instance[7][p], ca_adj))
                    
                    list_rs = [a*b for a,b in zip(list_r,list_s)]
                    list_rs = list(zip(list_rs, list(range(1,num_machine+1))))
                    list_rs = sorted(list_rs, reverse = True)

                    list_r_p = []
                    list_s_p = []
                    mach     = [] 
                    ca = instance[5][p]
                    
                    #print(ca)

                    for r in range(num_machine):

                        p = list_rs[r][1]-1
                        list_r_p.append(list_r[p])       # 製作 Ri list
                        list_s_p.append(list_s[p])       # 製作 Si list
                        
                        #print(list_r_p)
                        #print(list_s_p)
                                            
                        mach.append([])                  # machine set中安排一台台 machine 被 append 進來
                        mach[r].append(0)                # 初始的 benefit和 是 0
                        mach[r].append(ca * list_s_p[r]) # 初始的 capacity  是 K*Si
                                              
                    
                    mach_1 = mach
  
                    TotalBnft_2 = 0 
                    
                    for s in range(num_job):
                                                
                        Cs = list_c[s]
                        Ws = list_w[s]
                        Bs = list_b[s]
                        
                        w = 0
                        
                        #print(mach)
                        
                        mach_1 = sorted(mach_1, key=lambda x: x[1], reverse=True)  # 仍有 Ws台以上 machine 有餘裕

                        #print(mach_1)

                        #print(Cs)
                        #print(Ws)
                        #print(mach_1[Ws-1][1])
                        
                        if mach_1[Ws-1][1] >= Cs:              

                            for r in range(num_machine):

                                if (Cs <= mach[r][1] and w < Ws):               # 按 SiRi順序 assign，直到 Ws台 machine都完成

                                    mach[r][1] -= Cs
                                    mach[r][0] += Bs

                                    mach_1 = mach

                                    w += 1

                    for r in range(num_machine):                  
                        TotalBnft_2 += mach[r][0] * list_r_p[r] # 將所有 machine 的 benefit*quality factor 和加起來成為一組
                        
                        #print(mach[r][0])
                        #print(list_r_p[r])
                        #print(TotalBnft_2)

                    TotalBenefit_2 = np.append(TotalBenefit_2, TotalBnft_2) # 將 100組全 append在一起 
                    #print(TotalBenefit_2)


                # ====== Model 1 Objective value: ====== #

                list_M1            = list(TotalBenefit_1)

                # ====== Model 2 Objective value: ====== #

                list_M2            = list(TotalBenefit_2)
                
                #print(list_M1)
                #print(list_M2)

                list_ratio = list(map(lambda a, b: a / b, list_M2, list_M1))
                TotalBenefit_100_1 = sum(list_ratio) / len(list_ratio)
                TotalBenefit_ijk_sn.append(TotalBenefit_100_1)     # i*j*k*l 種 senario 迭代

                print('=========================' + str(i) + str(j) + str(k) + str(l) + '=========================') # 分隔           
            
print(TotalBenefit_ijk_sn)


CPLEX 12.8.0.0: optimal solution; objective 11480.67466
0 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 8464.445004
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 6504.686982
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 11345.58151
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 11336.25087
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12432.60767
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 8961.493979
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 11923.67327
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 9941.790692
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 9247.756994
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12571.90756
50 simp

CPLEX 12.8.0.0: optimal solution; objective 383036.9574
54 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 390642.3321
113 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 358324.4043
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 456738.2486
107 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 354038.8784
57 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 457384.0335
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 435648.9199
105 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 364147.1533
103 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 243473.1594
111 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 300327.441
101 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 401521.4332
105 simp

CPLEX 12.8.0.0: optimal solution; objective 2028.167998
102 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1677.726766
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2071.634068
98 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2387.280406
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1542.290963
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1681.940119
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2043.617532
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1623.502071
50 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2070.453724
100 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2267.826211
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2353.033433
101 simpl

CPLEX 12.8.0.0: optimal solution; objective 412186.6412
53 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 426949.0849
155 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 318023.033
160 dual simplex iterations (138 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 414603.333
58 dual simplex iterations (58 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 383726.8134
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 384731.5135
102 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 339261.6849
103 dual simplex iterations (50 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 401352.7548
89 dual simplex iterations (75 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 270380.8905
35 dual simplex iterations (32 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 371472.0538
127 dual simplex iterations (118 in phase I)
CPLEX 12.8.0.0: optimal soluti

CPLEX 12.8.0.0: optimal solution; objective 268219.3495
42 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 289794.8362
104 dual simplex iterations (95 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 419705.1507
64 dual simplex iterations (59 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 283908.1149
39 dual simplex iterations (39 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 339932.5444
122 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 417946.3549
91 dual simplex iterations (62 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 277315.1333
178 dual simplex iterations (141 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 334396.7666
99 dual simplex iterations (91 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 386942.5695
94 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 350866.7693
127 dual simplex iterations (75 in phase I)
CPLEX 12.8.0.0: optimal

...

CPLEX 12.8.0.0: optimal solution; objective 294504.681
268 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 288582.3697
261 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 290866.7257
277 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 286432.6832
246 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 288650.9931
264 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 294662.0537
242 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 271061.8346
247 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 275329.4423
243 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 260213.6454
259 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 304952.0651
268 dual simplex iterations (0 in phase I)
CPLEX 12.8.

CPLEX 12.8.0.0: optimal solution; objective 12667121.12
2382 dual simplex iterations (1444 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 14682927.42
2457 dual simplex iterations (1420 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13985973.81
2244 dual simplex iterations (1350 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13653728.05
3133 dual simplex iterations (2381 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 14401223.05
2121 dual simplex iterations (1509 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13077349.45
2469 dual simplex iterations (1699 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 15780301.59
2586 dual simplex iterations (1481 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13020769.01
2538 dual simplex iterations (1610 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 14271184.69
2951 dual simplex iterations (2035 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 14865356.48
2809 dual simplex

CPLEX 12.8.0.0: optimal solution; objective 78094.19462
9015 dual simplex iterations (7885 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 77622.22458
7982 dual simplex iterations (7069 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 88371.79962
6739 dual simplex iterations (5672 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 72023.94248
8161 dual simplex iterations (6893 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 75152.82233
7598 dual simplex iterations (6604 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 88880.8379
8640 dual simplex iterations (7511 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 106242.5283
8261 dual simplex iterations (7154 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 99103.70624
5409 dual simplex iterations (4351 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 84709.74764
8903 dual simplex iterations (7850 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 87102.24266
5844 dual simplex 

CPLEX 12.8.0.0: optimal solution; objective 85923.92249
7906 dual simplex iterations (7070 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 88865.81922
8482 dual simplex iterations (7466 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 80855.28002
9136 dual simplex iterations (8081 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 103934.37
8555 dual simplex iterations (7493 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 96422.66795
8795 dual simplex iterations (7553 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 93151.64065
7421 dual simplex iterations (6243 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 93280.67068
8651 dual simplex iterations (7405 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 92704.67403
7948 dual simplex iterations (6884 in phase I)
=========================(50, 500)ATM=========================
CPLEX 12.8.0.0: optimal solution; objective 831170.8736
8143 dual simplex iterations (7195 in phase I)
CPLEX 12.8.0

CPLEX 12.8.0.0: optimal solution; objective 864909.2861
7918 dual simplex iterations (6964 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1021639.848
6051 dual simplex iterations (5242 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 873076.8099
8302 dual simplex iterations (7430 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 825457.1979
8269 dual simplex iterations (7370 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 851067.4349
6216 dual simplex iterations (5336 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 777393.4508
8343 dual simplex iterations (7393 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 939922.3885
6753 dual simplex iterations (5791 in phase I)

"option abs_boundtol 1.1102230246251565e-16;"
or "option rel_boundtol 1.1102230246251565e-16;"
will change deduced dual values.

CPLEX 12.8.0.0: optimal solution; objective 919334.8152
8229 dual simplex iterations (7228 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 96

In [11]:
import pickle
with open("instance_list_HSQFCC_2_RS1to10.txt", "wb") as fp:
    pickle.dump(instance_list, fp)

with open("instance_list_HSQFCC_2_RS1to10.txt", "rb") as fp:
    instance_saved = pickle.load(fp)

In [12]:
print(instance_saved[0][7])

[[4.153004547673567, 8.263833155189417, 6.1966608051127, 9.767307397917179, 8.185728927071636], [7.935348835663174, 4.256778354934062, 3.035011800761021, 8.660602069026673, 3.5814892279373924], [8.508913897119188, 5.266609370885212, 5.498480289745131, 3.2737330924379644, 6.620686864724486], [1.136621684860426, 6.119477408861636, 3.4795708782128347, 5.564737807379541, 9.373267879944656], [6.289401528782534, 6.652160177351549, 2.872231193750432, 9.94216382634157, 6.106686452165946], [8.96511051923828, 4.559370965298616, 4.876251560458913, 7.172832586898776, 1.0596453309816292], [2.5226900037976865, 7.204869762715564, 9.659698129097094, 1.3362390937776594, 1.8785005383072155], [9.665478334483325, 4.6497074168598935, 1.3559730289402796, 7.480835825735853, 3.763679881352658], [3.7011905152429723, 9.845172198003713, 6.095397091938622, 7.636135149272753, 4.623424701117193], [8.631366359334631, 7.149918850974924, 1.6806815172004161, 6.335077465564034, 5.920135670936542], [6.358463681276385, 8.

In [13]:
tag = []
for i in machine_job:
    for l in cowork:
        for k in capacity:    
            for j in relation_bc:
                t = [i,l,k,j]
                tag.append(t)            

z = list(zip(tag, TotalBenefit_ijk_sn))
z

[([(5, 50), 'O', 'N', 'L'], 1.0),
 ([(5, 50), 'O', 'N', 'X'], 1.0),
 ([(5, 50), 'O', 'N', 'A'], 1.0),
 ([(5, 50), 'O', 'N', 'R'], 1.0),
 ([(5, 50), 'O', 'L', 'L'], 0.93992649680600915),
 ([(5, 50), 'O', 'L', 'X'], 0.9206826779215721),
 ([(5, 50), 'O', 'L', 'A'], 0.92870139325966972),
 ([(5, 50), 'O', 'L', 'R'], 0.90138603429947073),
 ([(5, 50), 'O', 'T', 'L'], 0.8957204038526132),
 ([(5, 50), 'O', 'T', 'X'], 0.90597897243688796),
 ([(5, 50), 'O', 'T', 'A'], 0.89301000613991777),
 ([(5, 50), 'O', 'T', 'R'], 0.89784773884744462),
 ([(5, 50), 'F', 'N', 'L'], 1.0),
 ([(5, 50), 'F', 'N', 'X'], 1.0),
 ([(5, 50), 'F', 'N', 'A'], 0.99999999999999989),
 ([(5, 50), 'F', 'N', 'R'], 1.0),
 ([(5, 50), 'F', 'L', 'L'], 0.93324878424259372),
 ([(5, 50), 'F', 'L', 'X'], 0.93711407923636658),
 ([(5, 50), 'F', 'L', 'A'], 0.91136208070186586),
 ([(5, 50), 'F', 'L', 'R'], 0.9036194850038558),
 ([(5, 50), 'F', 'T', 'L'], 0.92627624534739528),
 ([(5, 50), 'F', 'T', 'X'], 0.93707857903225633),
 ([(5, 50), 'F'